In [1]:
seed = 123

In [2]:
# import helper functions
from shutil import copyfile

copyfile(
    src = "/kaggle/input/bdc-p53/p53_helper_functions.py", 
    dst = "/kaggle/working/p53_helpers.py"
)

from p53_helpers import *

# load dataset (no NAs)
X, Y = load_p53_ds()

# sratified 80/20 split
X_train, X_test, Y_train, Y_test = split_p53(X, Y)

Using TensorFlow backend.



Import completed after 3.1 sec

CLASS RATIOS
Training set active classes:   121/24927 (0.485 %)
Test set active classes:       30/6232 (0.481 %)

MATRIX DIMENSIONS
TRAINING SET
. Features:   (24927, 5408)
. Classes:    (24927,)
TEST SET
. Features:   (6232, 5408)
. Classes:    (6232,)


In [3]:
def make_model(
    x, y,
    u1=256,
    d1=0.5,
    lr=0.001,
    smart_init=True
):
    
    # define metrics
    metrics = [
        mcc,
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc', curve="PR")
    ]
    
    if smart_init:
        n_active = y.sum()
        n_inactive = len(y) - n_active
        b0 = tf.keras.initializers.Constant(np.log([n_active / n_inactive]))
    else:
        b0=None # random initialization

    # build & compile
    model = keras.Sequential([
        keras.layers.Dense(u1, activation='relu', input_shape=(x.shape[-1], )),
        keras.layers.Dropout(d1),
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=b0)
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(lr=lr),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )
    
    return model

In [4]:
CV = 4

# to numpy arrays
X_train, Y_train = np.array(X_train), np.array(Y_train)

# init results df
results = pd.DataFrame(
    index=np.arange(5000), 
    columns=[
        "i", "fold", "n_remove", "scaler", "clip", "u1", "d1", "batch", "epoch", "patience", "lr",
        "precision", "recall", "auc", "mcc"
    ]
)

i = 0

SKF = StratifiedKFold(n_splits=CV, shuffle=True, random_state=seed)
folds = SKF.split(X_train, Y_train)

for f, (train_idx, val_idx) in enumerate(folds):
    
    print('\nFold ', f)
    
    X_t, Y_t = X_train[train_idx], Y_train[train_idx]
    X_v, Y_v = X_train[val_idx], Y_train[val_idx]

    for n_remove in [0]:
        
        if n_remove:
            rm = RemoveCorrelatedFeatures(n_remove=n_remove)
            X_t_ = rm.fit_transform(X_t)
            X_v_ = rm.transform(X_v)
            
        else:
            X_t_, X_v_ = X_t, X_v

        # scale
        scaler = StandardScaler()
        X_t_scaled = scaler.fit_transform(X_t_)
        X_v_scaled = scaler.transform(X_v_)
        scaler_name = scaler.__class__.__name__
        
        for clip in [None, 3, 5]:

            if clip:
                for x in [X_t_scaled, X_v_scaled]:
                    np.clip(x, -clip, clip, out=x)
            
            u1, d1 = 256, 0.5
            batch_size = 4096
            epoch = 75
            patience = 20

            for lr in [0.01, 0.005, 0.001]:

                early_stopping = tf.keras.callbacks.EarlyStopping(
                    monitor='val_mcc',
                    verbose=1,
                    patience=patience,
                    mode='max',
                    restore_best_weights=True
                )

                model = make_model(x=X_t_scaled, y=Y_t, lr=lr)

                out = model.fit(
                    X_t_scaled,
                    Y_t,
                    batch_size=batch_size,
                    epochs=epoch,
                    callbacks=[early_stopping],
                    validation_data=(X_v_scaled, Y_v)
                )

                summary = [i, f, n_remove, scaler_name, str(clip), u1, d1, batch_size, epoch, patience, lr]
                scores = [
                    out.history["val_" + m][-1]
                    for m in ["precision", "recall", "auc", "mcc"]
                ]
                summary.extend(scores)

                for col, value in zip(results.columns, summary):
                    results[col][i] = value

                i += 1


Fold  0
Train on 18695 samples, validate on 6232 samples
Epoch 1/75
18695/18695 [==============================] - 7s 365us/sample - loss: 2.6219 - mcc: 0.0649 - tp: 27.0000 - fp: 1824.0000 - tn: 16780.0000 - fn: 64.0000 - accuracy: 0.8990 - precision: 0.0146 - recall: 0.2967 - auc: 0.0095 - val_loss: 0.4418 - val_mcc: -8.1336e-04 - val_tp: 0.0000e+00 - val_fp: 1.0000 - val_tn: 6201.0000 - val_fn: 30.0000 - val_accuracy: 0.9950 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.0107
Epoch 2/75
18695/18695 [==============================] - 2s 117us/sample - loss: 0.5852 - mcc: -8.4604e-04 - tp: 0.0000e+00 - fp: 10.0000 - tn: 18594.0000 - fn: 91.0000 - accuracy: 0.9946 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0048 - val_loss: 0.6005 - val_mcc: 0.2268 - val_tp: 2.0000 - val_fp: 1.0000 - val_tn: 6201.0000 - val_fn: 28.0000 - val_accuracy: 0.9953 - val_precision: 0.6667 - val_recall: 0.0667 - val_auc: 0.0509
Epoch 3/75
18695/18695 [=============================

In [5]:
results = results.dropna(axis=0)
results

,i,fold,n_remove,scaler,clip,u1,d1,batch,epoch,patience,lr,precision,recall,auc,mcc
0,0,0,0,StandardScaler,None,256,0.5,4096,75,20,0.01,0.733333,0.366667,0.369028,0.527347
1,1,0,0,StandardScaler,None,256,0.5,4096,75,20,0.005,0.785714,0.366667,0.354943,0.543207
2,2,0,0,StandardScaler,None,256,0.5,4096,75,20,0.001,0.705882,0.4,0.449071,0.540245
3,3,0,0,StandardScaler,3,256,0.5,4096,75,20,0.01,0.857143,0.2,0.334897,0.425686
4,4,0,0,StandardScaler,3,256,0.5,4096,75,20,0.005,0.611111,0.366667,0.319881,0.490657
5,5,0,0,StandardScaler,3,256,0.5,4096,75,20,0.001,0.666667,0.4,0.510447,0.528053
6,6,0,0,StandardScaler,5,256,0.5,4096,75,20,0.01,0.705882,0.4,0.43861,0.549365
7,7,0,0,StandardScaler,5,256,0.5,4096,75,20,0.005,0.6875,0.366667,0.436673,0.506759
8,8,0,0,StandardScaler,5,256,0.5,4096,75,20,0.001,0.666667,0.4,0.47348,0.528053
9,9,1,0,StandardScaler,None,256,0.5,4096,75,20,0.01,0.5,0.4,0.353625,0.451157


In [6]:
results.to_csv("results.csv")